### JOIN Dataset - Exploratory - Exploratory

This Notebook contains the exploratory process of joining the two datasets.

Here you'll find the process to join the two files (NY Times Data Source and Population by County)

### Learnings

- FIPS might be on format SS for state and CCC for counties concatenated SSCCC to be able to join
- There are in total 61 counties without population data (46 with Unknown name)
- There are 15 counties name with Unknown name (Some of them repeated over different states)
- Using cumsum() function to calculate cumulative deaths/cases per day
- Documented issues related to columns being convert to object type when NaN values created


In [1]:
import pandas as pd
import numpy as np

In [13]:
url="https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv"
covidCounties = pd.read_csv(url, parse_dates=True, keep_default_na=False)

In [14]:
# Removing Puerto Rico since is not part of US satates
covidCounties = covidCounties[covidCounties['state'] !='Puerto Rico']

In [15]:
# Fix known issues to reconvert to Integer
covidCounties['deaths'] = covidCounties['deaths'].str.strip().astype('float').astype('Int64')

In [16]:
# Convert to date time for feature order
covidCounties['date'] = pd.to_datetime(covidCounties['date'])
# Trying to fix New York 
covidCounties.loc[covidCounties['county'] == 'New York City', 'fips']  = '36061'

In [17]:
covidCounties

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061,1,0
1,2020-01-22,Snohomish,Washington,53061,1,0
2,2020-01-23,Snohomish,Washington,53061,1,0
3,2020-01-24,Cook,Illinois,17031,1,0
4,2020-01-24,Snohomish,Washington,53061,1,0
...,...,...,...,...,...,...
1498302,2021-07-08,Sweetwater,Wyoming,56037,4820,40
1498303,2021-07-08,Teton,Wyoming,56039,3807,11
1498304,2021-07-08,Uinta,Wyoming,56041,2328,13
1498305,2021-07-08,Washakie,Wyoming,56043,928,26


In [39]:
# Importing data
url="https://www2.census.gov/programs-surveys/popest/datasets/2010-2019/counties/totals/co-est2019-alldata.csv"
countiesData = pd.read_csv(url, parse_dates=True, keep_default_na=False, encoding='ISO-8859-1')

In [40]:
# Using only data to preview some insights
countiesData = countiesData[countiesData['SUMLEV'] == 50][['SUMLEV','STATE','COUNTY','STNAME','CTYNAME','POPESTIMATE2019']]
countiesData.rename(columns={"POPESTIMATE2019": "population_2019"},  inplace = True)

In [41]:
# Filling fields with 0 to create FIPS format
countiesData['COUNTY'] = countiesData['COUNTY'].astype(str).str.zfill(3)
countiesData['STATE'] = countiesData['STATE'].astype(str).str.zfill(2)

In [42]:
# Creting fips with format SSCCC (5 digits)
countiesData['fips'] = countiesData['STATE'] +  countiesData['COUNTY']
countiesData.dtypes

SUMLEV              int64
STATE              object
COUNTY             object
STNAME             object
CTYNAME            object
population_2019     int64
fips               object
dtype: object

In [54]:
# Joining data
joined = covidCounties.merge(countiesData, how='left', on='fips')
joined['population_2019'] = joined['population_2019'].astype('Int64')
joined['deaths'] = joined['deaths'].astype('Int64')

In [23]:
# Find list of county with empty right join
cty = joined[['county','CTYNAME']].drop_duplicates()
cty['county'] = cty['county'] + ' County' 
cty[cty['CTYNAME']  != cty['CTYNAME'] ]

,county,CTYNAME
418,Unknown County,NaN
5634,Kansas City County,NaN
37793,St. Croix County,NaN
37794,St. John County,NaN
37795,St. Thomas County,NaN
159131,Bristol Bay plus Lake and Peninsula County,NaN
268820,Joplin County,NaN
327916,Saipan County,NaN
327917,Tinian County,NaN
357201,Yakutat plus Hoonah-Angoon County,NaN


In [24]:
# List all 61 counties without population data
cty = joined[['county','state','CTYNAME']].drop_duplicates()
cty['county'] = cty['county'] + ' County' 
cty[cty['CTYNAME']  != cty['CTYNAME'] ]

,county,state,CTYNAME
418,Unknown County,Rhode Island,NaN
1511,Unknown County,New Jersey,NaN
2265,Unknown County,Virgin Islands,NaN
2420,Unknown County,Guam,NaN
2926,Unknown County,Maine,NaN
...,...,...,...
327917,Tinian County,Northern Mariana Islands,NaN
357201,Yakutat plus Hoonah-Angoon County,Alaska,NaN
400232,Unknown County,South Carolina,NaN
763450,Unknown County,Oregon,NaN


In [55]:
# Ordering to create comulative
joined = joined[['date','fips','county','state','cases','deaths','population_2019']]
joined.sort_values(['date']).reset_index(drop=True)

,date,fips,county,state,cases,deaths,population_2019
0,2020-01-21,53061,Snohomish,Washington,1,0,822083
1,2020-01-22,53061,Snohomish,Washington,1,0,822083
2,2020-01-23,53061,Snohomish,Washington,1,0,822083
3,2020-01-24,17031,Cook,Illinois,1,0,5150233
4,2020-01-24,53061,Snohomish,Washington,1,0,822083
...,...,...,...,...,...,...,...
1464316,2021-07-08,21117,Kenton,Kentucky,17284,213,166998
1464317,2021-07-08,21119,Knott,Kentucky,1165,24,14806
1464318,2021-07-08,21121,Knox,Kentucky,3891,47,31145
1464319,2021-07-08,21125,Laurel,Kentucky,7612,81,60813


In [56]:
# Create comulative by fips, once they are ordered
joined["comulative_cases"] = joined.groupby(['fips'])['cases'].cumsum(axis=0)
joined["comulative_deaths"] = joined.groupby(['fips'])['deaths'].cumsum(axis=0)

In [57]:
# Reconvert
joined["comulative_deaths"] =joined["comulative_deaths"].astype('Int64')
joined.dtypes

date                 datetime64[ns]
fips                         object
county                       object
state                        object
cases                         int64
deaths                        Int64
population_2019               Int64
comulative_cases              int64
comulative_deaths             Int64
dtype: object

In [58]:
# Prove comulatives is working
joined[joined['fips'] == '53061']

,date,fips,county,state,cases,deaths,population_2019,comulative_cases,comulative_deaths
0,2020-01-21,53061,Snohomish,Washington,1,0,822083,1,0
1,2020-01-22,53061,Snohomish,Washington,1,0,822083,2,0
2,2020-01-23,53061,Snohomish,Washington,1,0,822083,3,0
4,2020-01-24,53061,Snohomish,Washington,1,0,822083,4,0
7,2020-01-25,53061,Snohomish,Washington,1,0,822083,5,0
...,...,...,...,...,...,...,...,...,...
1451493,2021-07-04,53061,Snohomish,Washington,40330,605,822083,8315400,156060
1454660,2021-07-05,53061,Snohomish,Washington,40330,605,822083,8355730,156665
1457827,2021-07-06,53061,Snohomish,Washington,40437,608,822083,8396167,157273
1460994,2021-07-07,53061,Snohomish,Washington,40493,610,822083,8436660,157883
